In [22]:
# needed imports for this notebook
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sweetviz as sv



In [107]:
# load csv file ad pandas dataframe
df_original = pd.read_csv('D:\FU WINFO\I\Applied Analytics\kip_sdm_syn.csv')

In [45]:
#run sweetviz on df to generate report
report_original = sv.analyze(df_original)


                                             |          | [  0%]   00:00 -> (? left)

In [46]:
# show sweetviz report
report_original.show_html('D:\FU WINFO\I\Applied Analytics\kip_sdm_syn_report_oiriginal.html')

Report D:\FU WINFO\I\Applied Analytics\kip_sdm_syn_report_oiriginal.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [121]:
# ignore columns "fall-while_stay" and "fall-while_transfer" as they are already included in outcome and not relevant for general risk assessment
columns_to_drop = ["fall-while_stay", "fall-while_transfer"]
df = df_original.drop(columns=columns_to_drop)

In [122]:
# if walk-impairment empty and (walking_aid == 1 or walk_jones not empty), then set walk-impairment to 1
# Iterate through rows and update 'walk-impairment' based on the condition
for index, row in df.iterrows():
    if pd.isna(row['walk-impairment']):  # Check if 'walk-impairment' is NaN
        if row['walking_aid'] == 1:  # Check if 'walking_aid' is 1
            df.at[index, 'walk-impairment'] = 1  # Set 'walk-impairment' to 1
        elif pd.notna(row['walk-jones']):  # Check if 'walk-jones' is not NaN
            df.at[index, 'walk-impairment'] = 1  # Set 'walk-impairment' to 1


In [123]:
# convert bed_mobility-jones, transfer and walk-jones to one-hot encoded columns
# Assuming `columns_to_transform` is the list of columns you want to transform
columns_to_transform = ['bed_mobility-jones', 'transfer', 'walk-jones']  # Replace with your column names

for col in columns_to_transform:
    # Create one-hot encoded columns for each distinct value in the column
    dummies = pd.get_dummies(df[col], prefix=col)
    # Add the new columns to the DataFrame
    df = pd.concat([df, dummies], axis=1)
    # Drop the original column
    df = df.drop(columns=[col])

In [124]:
# Convert all boolean columns to numeric (False -> 0, True -> 1, NA -> NaN)
bool_columns = [
    col for col in df.columns 
    if df[col].dropna().isin([True, False]).all()
]
df[bool_columns] = df[bool_columns].astype('Int64')


In [126]:
# 2. decubitus-risk & fall-risk: adjust columns having "+" and "++" values to numerical values
columns_to_modify = ["decubitus-risk", "fall-risk"] 

value_mapping = {
    '0': 0,
    '+': 1,
    '++': 2
}

df[columns_to_modify] = df[columns_to_modify].replace(value_mapping)

C:\Users\Veit\AppData\Local\Temp\ipykernel_22120\1550453215.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[columns_to_modify] = df[columns_to_modify].replace(value_mapping)


In [127]:
# sex: transform to numeric binary value 

df['is_male'] = df['sex'].apply(lambda x: 1 if x == 'male' else 0)
# Drop the 'sex' column
df = df.drop(columns=['sex'])


In [128]:
# decubitus-admission, medical_items-A_V_system: drop since only distinct value is "False"
columns_to_drop = ["decubitus-admission", "medical_items-A_V_system"]
df = df.drop(columns=columns_to_drop)

In [ ]:
# replace NA with nan (otherwise sweetviz fails)
df = df.replace({pd.NA: np.nan})

In [132]:
print(df.dtypes)

fallen                                                Int64
id                                                    int64
age                                                   int64
procedure                                            object
diagnosis                                            object
c_zriserh1                                            Int64
decubitus-at_the_moment                               Int64
bed_mobility-impairment                               Int64
bed_mobility-skin_condition                           Int64
transfer-impairment                                   Int64
medical_items                                         Int64
medical_items-arm_or_leg_splint                       Int64
medical_items-plaster_or_neck_brace                   Int64
medical_items-compression_stocking                    Int64
medical_items-ventilation_mask                        Int64
medical_items-O2_glasses                              Int64
medical_items-orthesis                  

In [ ]:
#run sweetviz on df to generate report
report = sv.analyze(df)


                                             |          | [  0%]   00:00 -> (? left)

In [43]:
# show sweetviz report
report.show_html('D:\FU WINFO\I\Applied Analytics\kip_sdm_syn_report.html')

Report D:\FU WINFO\I\Applied Analytics\kip_sdm_syn_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create a copy of the DataFrame to avoid modifying the original
df_encoded = df.copy()

# Encode non-numerical columns using LabelEncoder
for column in df_encoded.select_dtypes(include=["object", "category"]).columns:
    le = LabelEncoder()
    df_encoded[column] = le.fit_transform(df_encoded[column].astype(str))  # Convert to string before encoding

# Identify columns with mixed types and convert them to strings before encoding
for column in df_encoded.columns:
    if df_encoded[column].apply(type).nunique() > 1:  # Check for mixed types
        df_encoded[column] = df_encoded[column].astype(str)

# Calculate the correlation of each column with the binary outcome variable 'fallen'
correlations = df_encoded.corr()["fallen"].drop("fallen")

# Sort the correlations in descending order
sorted_correlations = correlations.sort_values(ascending=False)

# Format and display the sorted correlations as a list
for column, correlation in sorted_correlations.items():
    print(f"{column}: {correlation}")

In [117]:
# Print all unique values in the 'sex' column
unique_values = df_original['walk-jones'].unique()
print(f"Unique values in 'sex': {unique_values}, length: {len(unique_values)}")

Unique values in 'sex': ['2' nan '1' '4B' '3' '4A'], length: 6
